In [162]:
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import os

In [350]:
forder= os.listdir('1월 데이터')

forder.sort()
# print(forder)

# exit()
weather_by_region = {}
for file in forder:
    csv = pd.read_csv('1월 데이터/' + file, encoding='CP949')
    csv = csv.drop(['지점명'], axis=1)
    groups = csv.groupby(csv.columns[0])
    for i in groups:
        if weather_by_region.get(i[0]) is not None:
            weather_by_region[i[0]].append(list(i))
        else:
            weather_by_region[i[0]] = list(i)

In [351]:
type(weather_by_region[310][1])

for i in weather_by_region:
    region_data=weather_by_region[i]
    
    concated_data = region_data[1]
    size = len(region_data)
    for j in range(2,size):
        concated_data = pd.concat([concated_data, region_data[j][1]])
    
    region_data = concated_data.values
    size=len(region_data)
    # print(region_data[567])
    result = []
    sum = np.zeros(14).astype(float)
    
    for j in range(size):
        row_data = region_data[j][2:].astype(float)
        sum += row_data
        if (j+1) % 60 == 0:
            result.append(sum/60)
            sum = np.zeros(14)

    weather_by_region[i]=result

In [353]:
weather_by_region[310][2]

array([  -2.585     ,    0.        ,    0.        ,    0.        ,
        275.57833333,    0.        ,    3.45833333,    0.        ,
       1027.78833333,    0.        , 1029.58833333,    0.        ,
         21.28      ,    0.        ])

In [378]:
powers = []
forder = os.listdir('2022_01_energy/2022_01')

forder=sorted(forder,key=lambda x:int(x.split('.')[0]))
# print(forder)

for file in forder:
    xlsx = pd.read_excel('2022_01_energy/2022_01/'+file, engine='openpyxl', skiprows=range(4))
    xlsx = xlsx.iloc[:-1, :]  #row remove
    powers.append(xlsx)
    
# powers

In [381]:
# 각 데이터프레임의 인덱스를 재설정
reset_powers = [df.reset_index(drop=True) for df in powers]

# 행 기준 : axis = 0
combined_powers = pd.concat(reset_powers, axis=1)

powers_np = combined_powers.to_numpy()


In [382]:
powers_np

array([[datetime.datetime(2022, 1, 1, 1, 0), 0.0,
        datetime.datetime(2022, 1, 2, 1, 0), 0.0,
        datetime.datetime(2022, 1, 3, 1, 0), 0.0,
        datetime.datetime(2022, 1, 4, 1, 0), 0.0,
        datetime.datetime(2022, 1, 5, 1, 0), 0.0,
        datetime.datetime(2022, 1, 6, 1, 0), 0.0,
        datetime.datetime(2022, 1, 7, 1, 0), 0.0,
        datetime.datetime(2022, 1, 8, 1, 0), 0.0,
        datetime.datetime(2022, 1, 9, 1, 0), 0.0,
        datetime.datetime(2022, 1, 10, 1, 0), 0.0,
        datetime.datetime(2022, 1, 11, 1, 0), 0.0,
        datetime.datetime(2022, 1, 12, 1, 0), 0.0,
        datetime.datetime(2022, 1, 13, 1, 0), 0.0,
        datetime.datetime(2022, 1, 14, 1, 0), 0.0,
        datetime.datetime(2022, 1, 15, 1, 0), 0.0,
        datetime.datetime(2022, 1, 16, 1, 0), 0.0,
        datetime.datetime(2022, 1, 17, 1, 0), 0.0,
        datetime.datetime(2022, 1, 18, 1, 0), 0.0,
        datetime.datetime(2022, 1, 19, 1, 0), 0.0,
        datetime.datetime(2022, 1, 20, 1

In [362]:
import torch

class CustomDataset(Dataset):
    def __init__(self, weather, energy):
        self.weather = weather # 일자별 df 배열
        print(len(self.weather[310]))
        self.energy = energy # 에너지 리스트 일자 상관없음.
        print(len(self.energy))
    
    def __len__(self):
        return len(self.energy)
    
    def __getitem__(self, idx):
        weathers = {}
        
        for i in self.weather:
            weathers[i]=self.weather[i]
        
        return self.energy[idx],self.weather

dataset = CustomDataset(weather_by_region, powers)
dataloader = DataLoader(dataset, batch_size=10, shuffle=False)

743
31


In [357]:
import sys
# np.set_printoptions(threshold=sys.maxsize)

for data,result in dataloader:
    # print(np.array(data[0]))
    print(f'data : {data.shape} result : {result.shape}')

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas.core.frame.DataFrame'>

In [ ]:
from torch import nn
from torch import optim


class predictModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(predictModel, self).__init__()
        self.dense = nn.Linear(76, 1)
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self,x):
        # print("size of x: ", x) # torch.Size([10, 23, 76, 14])
        x = x.permute(0,1,3,2) # x.shape: (10, 23, 14, 76)
        x = self.dense(x) # x.shape: (10, 23, 14, 1)
        x = x.squeeze(-1) # x.shape: (10, 23, 14)
        out, _ = self.rnn(x)  # out shape: (batch_size, seq_len, hidden_size)
        out = self.fc(out[:, -1, :])  # out shape: (batch_size, output_size)
        return out


In [ ]:
def train(model):
    criterion = nn.MSELoss()
    # 옵티마이저 정의
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    for data, result in dataloader:
        optimizer.zero_grad()  # 경사 초기화
        # print(data.shape)

        outputs = model(data)  # 순전파
        print(f'outputs.shape : {outputs.shape} result.shape : {result.shape}')
        loss = criterion(outputs, result)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트
        print(f'train loss: {loss.item():.4f}')
    

In [ ]:
epoch=100
model = predictModel(14, 128,23)

for i in range(epoch):
    train(model)

outputs.shape : torch.Size([10, 23]) result.shape : torch.Size([10, 23])
train loss: nan


/home/user/anaconda3/lib/python3.9/site-packages/torch/autograd/__init__.py:173: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


outputs.shape : torch.Size([10, 23]) result.shape : torch.Size([10, 23])
train loss: nan
outputs.shape : torch.Size([10, 23]) result.shape : torch.Size([10, 23])
train loss: nan
outputs.shape : torch.Size([1, 23]) result.shape : torch.Size([1, 23])
train loss: nan
outputs.shape : torch.Size([10, 23]) result.shape : torch.Size([10, 23])
train loss: nan
outputs.shape : torch.Size([10, 23]) result.shape : torch.Size([10, 23])
train loss: nan
outputs.shape : torch.Size([10, 23]) result.shape : torch.Size([10, 23])
train loss: nan
outputs.shape : torch.Size([1, 23]) result.shape : torch.Size([1, 23])
train loss: nan
outputs.shape : torch.Size([10, 23]) result.shape : torch.Size([10, 23])
train loss: nan
outputs.shape : torch.Size([10, 23]) result.shape : torch.Size([10, 23])
train loss: nan
outputs.shape : torch.Size([10, 23]) result.shape : torch.Size([10, 23])
train loss: nan
outputs.shape : torch.Size([1, 23]) result.shape : torch.Size([1, 23])
train loss: nan
outputs.shape : torch.Size(

KeyboardInterrupt: 